In [91]:
''' fd=open('big.txt','r')
txt=fd.read()
fd.close() '''

import re
import pandas as pd


## Finding Unique Words

In [12]:
with open('big.txt','r') as fd:
    lines=fd.readlines()
    words=[]
    for line in lines:
        words += re.findall('\w+',line.lower())

print(len(words))
vocab = set(words)
print(len(vocab))

<>:5: SyntaxWarning: invalid escape sequence '\w'
<>:5: SyntaxWarning: invalid escape sequence '\w'
C:\Users\Dell\AppData\Local\Temp\ipykernel_37876\1453892446.py:5: SyntaxWarning: invalid escape sequence '\w'
  words += re.findall('\w+',line.lower())


1115585
32198


### Finding Probability Distribution

In [13]:
words.count('The')

0

In [14]:
words.count('the')

79809

In [18]:
word_prob={}

for a in vocab:
    word_prob[a] = float(words.count(a)/len(words))
    
    
    

In [19]:
len(word_prob)

32198

## Spell Checking Architecture

In [20]:
word_prob['sherlock']

9.053545897443941e-05

### Text Preprocessing

Splitting

In [81]:
def split(w):
    
    parts=[]

    for i in range(len(w)+1):  
       parts += [(w[:i],w[i:])]
    
    return parts

split('OM')

[('', 'OM'), ('O', 'M'), ('OM', '')]

1) Delete

   'loave' -> 'love'

In [82]:
def delete(w):
    
    output=[]
    for l,r in split(w):
        output.append(l + r[1:])
    return output

delete('Heallo')

['eallo', 'Hallo', 'Hello', 'Healo', 'Healo', 'Heall', 'Heallo']

2. Swap

   lvoe -> love

In [83]:
def swap(w):
    
    output=[]
    for l,r in split(w):
        if(len(r)>1):
            output.append(l + r[1] + r[0] + r[2:] )
    return output

swap('OM')

['MO']

3. Replacement

   lave -> love

In [84]:
def replace(w):

    output=[]
    alphabets = 'abcdefghijklmnopqrstuvwxyz'
    for l,r in split(w):
        for char in alphabets:
            output.append(l+char+r[1:])
    return output

len(replace('lave'))


130

4. Insert

   lve -> love

In [85]:
def insert(w):
    
    output=[]
    alphabets = 'abcdefghijklmnopqrstuvwxyz'
    
    for l,r in split(w):
        for c in alphabets:
            output.append(l+c+r)
    return output
    
insert('lve')

['alve',
 'blve',
 'clve',
 'dlve',
 'elve',
 'flve',
 'glve',
 'hlve',
 'ilve',
 'jlve',
 'klve',
 'llve',
 'mlve',
 'nlve',
 'olve',
 'plve',
 'qlve',
 'rlve',
 'slve',
 'tlve',
 'ulve',
 'vlve',
 'wlve',
 'xlve',
 'ylve',
 'zlve',
 'lave',
 'lbve',
 'lcve',
 'ldve',
 'leve',
 'lfve',
 'lgve',
 'lhve',
 'live',
 'ljve',
 'lkve',
 'llve',
 'lmve',
 'lnve',
 'love',
 'lpve',
 'lqve',
 'lrve',
 'lsve',
 'ltve',
 'luve',
 'lvve',
 'lwve',
 'lxve',
 'lyve',
 'lzve',
 'lvae',
 'lvbe',
 'lvce',
 'lvde',
 'lvee',
 'lvfe',
 'lvge',
 'lvhe',
 'lvie',
 'lvje',
 'lvke',
 'lvle',
 'lvme',
 'lvne',
 'lvoe',
 'lvpe',
 'lvqe',
 'lvre',
 'lvse',
 'lvte',
 'lvue',
 'lvve',
 'lvwe',
 'lvxe',
 'lvye',
 'lvze',
 'lvea',
 'lveb',
 'lvec',
 'lved',
 'lvee',
 'lvef',
 'lveg',
 'lveh',
 'lvei',
 'lvej',
 'lvek',
 'lvel',
 'lvem',
 'lven',
 'lveo',
 'lvep',
 'lveq',
 'lver',
 'lves',
 'lvet',
 'lveu',
 'lvev',
 'lvew',
 'lvex',
 'lvey',
 'lvez']

# Prediction

#### Combining possible words

In [86]:
def edit(word):
    return set(insert(word)+delete(word)+swap(word)+replace(word))

#### Predicting the Word

In [120]:
def spell_check(word,count=5):
    output=[]  
    suggested_words=edit(word)
    for i in suggested_words:
        if i in word_prob.keys():
           output.append([i, word_prob[i]])
    list = (pd.DataFrame(output,columns=['word','prob']).sort_values(by='prob',ascending=False).head(count)['word'].values)
    return list

In [121]:
spell_check('tha')

array(['the', 'that', 'than', 'tea', 'ha'], dtype=object)

In [139]:
spell_check('fameli')

array([], dtype=object)

# Prediction(Level-2)

In [137]:
def edit_2(word, count=5):
    output2=[]
    suggested2=[]
    
    for e1 in edit(word):
        output2 += edit(e1)
    output2 += edit(word)
    
    output2 = set(output2)
    
    for wrd in output2:
        if wrd in word_prob.keys():
            suggested2.append([wrd,word_prob[wrd]])
    list2 = (pd.DataFrame(suggested2,columns=['word','prob']).sort_values(by='prob',ascending=False).head(count)['word'].values) 
    return list2
        

In [138]:
edit_2('loave')

array(['have', 'love', 'gave', 'leave', 'late'], dtype=object)

In [140]:
edit_2('fameli')

array(['family', 'namely', 'fame', 'camelia', 'camel'], dtype=object)